In [22]:
import sys
sys.path.append("/home/rossidaniel/Documenti/bin-packing/Python/")
sys.path.append("/home/rossidaniel/Documenti/bin-packing/")
sys.path.append("/home/rossidaniel/Dropbox")
sys.path.append("/home/rossidaniel/Documenti/bin-packing/Python")

In [23]:
import Bin_Packing_Py as BP
import BP_print
import BP_plot
import BP_utilities
import HA_routines
import BP_instances

import matplotlib
%matplotlib nbagg
matplotlib.rcParams['figure.figsize'] = [6,3.5]

from ortools.linear_solver import pywraplp
import random
import time
import csv

import pandas as pd
from mail import send_mail
from Test_BP_G import euristic
import multiprocessing

In [24]:
class istanza:
    def __init__(self,int1,int2,ist_id):
        self.index=ist_id
        self.width=int1
        self.depth=int2

class intervallo:
    def __init__(self,a,b):
        self.a=a
        self.b=b

In [25]:
def tc_sfida(ist):
    item_list = []
    bin_list = []

    bin_list.append(BP.Bin(2.70, 50, 2.45))
    bin_list.append(BP.Bin(0, 0, 0))

    n_items =random.randint(3,6)

    for i in range(n_items):
        w=random.uniform(ist.width.a,ist.width.b)
        d=random.uniform(ist.depth.a,ist.depth.b)
        h=random.uniform(0.5,1.5)
        item = BP.Item(i,h,d,w)
        item.stackable = random.randint(0,0)
        item.order_id = random.randint(0,3)
        item.weight = 4
        item_list.append(item)

    return item_list, bin_list

In [26]:
class model_bin_packing:
    def __init__(self,item_list,bin_list):
        self.bp = BP.BinPack()
        self.bp.items.extend(item_list)
        self.bp.bins.extend(bin_list)
        
    def buildModel(self,amount_time=300000):
        solver = pywraplp.Solver('Model2DR', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
        #importo dimensioni dei pacchi e del camion
        n=len(self.bp.items)

        w =[self.bp.items[i].width for i in range(n)]
        d =[self.bp.items[i].depth for i in range(n)]
        h =[self.bp.items[i].height for i in range(n)]
        M=sum([max(w[i],d[i]) for i in range(n)])

        W =self.bp.bins[0].width
        D =solver.NumVar(0,M,"D")
        H =self.bp.bins[0].height
        
        Mw=W+M
        Md=self.bp.bins[0].depth+M
        Mz=H+max(h)
                
        # definisco le variabili
        l =[[solver.BoolVar("l%d%d" % (i,j)) for i in range(n)] for j in range(n)]
        b =[[solver.BoolVar("b%d%d" % (i,j)) for i in range(n)] for j in range(n)]
        r =[solver.BoolVar("r%d" % (i)) for i in range(n)]

        x =[solver.NumVar(0,solver.infinity(),"x%d" % i) for i in range(n)]
        y =[solver.NumVar(0,solver.infinity(),"y%d" % i) for i in range(n)]


        #definisco i constraints
        for i in range(n):
            for j in range(n):
                if(i < j):
                    solver.Add(l[i][j] + l[j][i] + b[i][j] + b[j][i] >= 1)                           #(1)
                if(i != j):
                    solver.Add(x[i] - x[j] + Mw * l[i][j] <= Mw - w[i]*(1-r[i]) - d[i]*r[i])         #(2)
                    solver.Add(y[i] - y[j] + Md * b[i][j] <= Md - d[i]*(1-r[i]) - w[i]*r[i])         #(3)
            solver.Add(x[i] + w[i]*(1-r[i]) + d[i]*r[i] <= W)                                        #(4)
            solver.Add(y[i] + d[i]*(1-r[i]) + w[i]*r[i] <= D)                                        #(5)

        #funzione obiettivo
        objective = solver.Objective()
        objective.SetCoefficient(D,1)
        objective.SetMinimization()

        #soluzione
        solver.SetTimeLimit(amount_time)
        t=time.time()
        solver.Solve()
        t =time.time()-t

        s = BP.Solution(self.bp)
        for i in range(len(self.bp.items)):
            rt = False if r[i].solution_value() == 0 else True
            s.pushItem(self.bp.items[i],0,BP.Point(round(x[i].solution_value(),3),round(y[i].solution_value(),3),0),rt,False)
        return s,t,n
    
    def BuildSolution(self,s):
        print s
    
    def plotSolution(self,s):
        BP_plot.plotSolution(s)
    
    def plot3Dsolution(self,s):
        BP_plot.plot3DSolution(s,0)

    def feasible(self):
        return self.bp.bins[0].height*self.bp.bins[0].width >= sum([self.bp.items[i].width*self.bp.items[i].height for i in range(len(self.bp.items))])

In [27]:
def calculate_result(istance,n_model,p_result):
    items,bins=tc_sfida(istance)
    model = model_bin_packing(items,bins)
    s,t,n=model.buildModel()
    s.computeObj()
    mod_obj =round(s.obj,2)
    eu_obj =round(euristic(items,bins),2)
    Time =round(t,3)
    n_items =n
    p_result[n_model]=[mod_obj,eu_obj,Time,n_items]
    

In [28]:
ist = []
df = pd.read_excel('istanza.xlsx')
for i in range(len(df)):
    ist.append(istanza(intervallo(df.w_a[i],df.w_b[i]),intervallo(df.d_a[i],df.d_b[i]),i))

In [29]:
tot_df = pd.DataFrame({'n_items': [], 'Eu_obj': [],'Mod_obj': [], 'Time': []})
tot_df = tot_df[['n_items', 'Eu_obj', 'Mod_obj', 'Time']]

In [30]:
tm=time.time()
models_x_istance = 100
for i in ist:
    manager = multiprocessing.Manager()
    p_result = manager.dict()
    jobs = []

    for j in range(models_x_istance):
        p = multiprocessing.Process(target=calculate_result, args=(i,j,p_result))
        jobs.append(p)
        p.start()
    
    for proc in jobs:
        proc.join()
    result=p_result.values()
    
    mod_obj = [n_it[0] for n_it in result]
    eu_obj = [n_it[1] for n_it in result]
    Time = [n_it[2] for n_it in result]
    n_items = [n_it[3] for n_it in result]
    
    dimensioni =pd.DataFrame({'n_items': n_items, 'Eu_obj': eu_obj,'Mod_obj': mod_obj, 'Time': Time})
    dimensioni = dimensioni[['n_items', 'Eu_obj', 'Mod_obj', 'Time']]

    dimensioni.to_excel('result'+str(i.index)+'.xlsx', encoding='utf8')
    tot_df =tot_df.append(dimensioni,ignore_index=True)
tot_df.to_excel('result_tot.xlsx', encoding='utf8')
print time.time()-tm

1453.98515201


In [18]:
send_mail()

In [19]:
tot_df

,n_items,Eu_obj,Mod_obj,Time
0,5.0,1.49,1.42,1.331
1,5.0,1.24,1.18,0.811
2,3.0,0.98,0.98,0.018
3,3.0,0.82,0.82,0.011
4,4.0,1.39,1.39,0.197
5,3.0,0.88,0.88,0.009
6,5.0,1.58,1.35,0.824
7,4.0,1.38,1.37,0.305
8,4.0,1.55,1.24,0.179
9,5.0,1.52,1.38,0.828
